In [4]:
import polars as pl
import pandas as pd

In [37]:
data = {
    'zug_id': [1]*7,
    'betriebstag': ['2022-01-01']*7,
    'zuglauf_id': [1]*3 + [2]*4,
    'halt_id': [1, 2, 3] + [1, 2, 4, 5],
}
df = pd.DataFrame(data)
df

,zug_id,betriebstag,zuglauf_id,halt_id
0,1,2022-01-01,1,1
1,1,2022-01-01,1,2
2,1,2022-01-01,1,3
3,1,2022-01-01,2,1
4,1,2022-01-01,2,2
5,1,2022-01-01,2,4
6,1,2022-01-01,2,5


In [38]:
df['shift'] = (
    df
    .sort_values(['betriebstag', 'zuglauf_id', 'halt_id'])
    .groupby(['betriebstag', 'zug_id'])
    ['halt_id']
    .shift(1)
)
df['shift2'] = df['halt_id'] - df['shift'] - 1
df['shift2'] = df[df['shift2'] > 0]['shift2']
df['shift2'].fillna(0, inplace=True)
df['shift2'] = (
    df
    .sort_values(['betriebstag', 'zug_id'])
    .groupby(['betriebstag', 'zug_id'])
    ['shift2']
    .cumsum()
)
df

,zug_id,betriebstag,zuglauf_id,halt_id,shift,shift2
0,1,2022-01-01,1,1,NaN,0.0
1,1,2022-01-01,1,2,1.0,0.0
2,1,2022-01-01,1,3,2.0,0.0
3,1,2022-01-01,2,1,3.0,0.0
4,1,2022-01-01,2,2,1.0,0.0
5,1,2022-01-01,2,4,2.0,1.0
6,1,2022-01-01,2,5,4.0,1.0


In [39]:
df['halt_id_fixed'] = (
    df
    .sort_values(['betriebstag', 'zug_id'])
    .groupby(['betriebstag', 'zug_id'])
    .cumcount() + 1
)
df

,zug_id,betriebstag,zuglauf_id,halt_id,shift,shift2,halt_id_fixed
0,1,2022-01-01,1,1,NaN,0.0,1
1,1,2022-01-01,1,2,1.0,0.0,2
2,1,2022-01-01,1,3,2.0,0.0,3
3,1,2022-01-01,2,1,3.0,0.0,4
4,1,2022-01-01,2,2,1.0,0.0,5
5,1,2022-01-01,2,4,2.0,1.0,6
6,1,2022-01-01,2,5,4.0,1.0,7


In [40]:
df['halt_id_fixed'] += df['shift2']
df

,zug_id,betriebstag,zuglauf_id,halt_id,shift,shift2,halt_id_fixed
0,1,2022-01-01,1,1,NaN,0.0,1.0
1,1,2022-01-01,1,2,1.0,0.0,2.0
2,1,2022-01-01,1,3,2.0,0.0,3.0
3,1,2022-01-01,2,1,3.0,0.0,4.0
4,1,2022-01-01,2,2,1.0,0.0,5.0
5,1,2022-01-01,2,4,2.0,1.0,7.0
6,1,2022-01-01,2,5,4.0,1.0,8.0
